In [67]:
from ZW_model import PSI
from ZW_utils import std_classes,dataloading
from ZW_dataset import PSI_Dataset

classes = std_classes
data_split_ratio = 0.85
batch_size = 100
max_epochs = 100
learning_rate = 1e-3
block_size = 22
n_embd = 32  # 32
n_head = 4  # 4
n_layer = 2  # 2
dropout = 0.1  # 0.1
vocab_size = len(classes)
model = PSI(vocab_size, n_embd, n_head, n_layer, block_size, dropout)

In [68]:
import numpy as np
from config import DATA_DIRECTORY
from torch.utils.data import DataLoader
import torch

# layouts = np.load(DATA_DIRECTORY/"v22DF_m2_sorted_layouts.npy", allow_pickle=True)
# results = np.load(DATA_DIRECTORY/"v22DF_m2_sorted_results.npy", allow_pickle=True)
layouts = np.load("GPT_NA/initial_10k.npy", allow_pickle=True)
results = np.load("GPT_NA/results_initial_10k.npy", allow_pickle=True)
l2 = []
r2 = []
for i,r in enumerate(results):
    if r > 0 and r <200:
        l2.append(layouts[i])
        r2.append(r)
layouts = np.asanyarray(l2)
results = np.asanyarray(r2)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
data = PSI_Dataset(layouts, results, classes,block_size,training_type="augmented")
training_set, validation_set = torch.utils.data.random_split(data, [int(data_split_ratio*len(data)), len(data)-int(data_split_ratio*len(data))])
train_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validation_set, batch_size=batch_size, shuffle=True)

for epoch in range(max_epochs):
    model.train()
    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        y_pred = model(x)
        y_pred = y_pred.view(y.size(0),block_size)
        y = y.to(torch.float32)
        loss = torch.nn.functional.mse_loss(y_pred, y)
        loss.backward()
        optimizer.step()
    val_loss = 0
    with torch.no_grad():
        model.eval()
        for i, (x, y) in enumerate(val_loader):
            y_pred = model(x)
            y_pred = y_pred.view(y.size(0),block_size)
            y = y.to(torch.float32)
            loss = torch.nn.functional.mse_loss(y_pred, y)
            val_loss += loss.item()
        val_loss /= len(val_loader)
    print(f"Epoch {epoch} - Validation loss: {val_loss}")

Designs in the dataset: 909
Data augmented: 11005
Input shape: torch.Size([11914, 22]) Output shape: torch.Size([11914, 22])
Epoch 0 - Validation loss: 26289.860894097223
Epoch 1 - Validation loss: 23918.026258680555
Epoch 2 - Validation loss: 20908.07693142361
Epoch 3 - Validation loss: 17507.637803819445
Epoch 4 - Validation loss: 13993.137803819445
Epoch 5 - Validation loss: 10617.104437934027
Epoch 6 - Validation loss: 7603.814344618056
Epoch 7 - Validation loss: 5118.548692491319
Epoch 8 - Validation loss: 3233.353529188368
Epoch 9 - Validation loss: 1924.7706095377605
Epoch 10 - Validation loss: 1103.23341539171
Epoch 11 - Validation loss: 639.1006401909722
Epoch 12 - Validation loss: 406.89385816786023
Epoch 13 - Validation loss: 303.913820054796
Epoch 14 - Validation loss: 264.07833777533637
Epoch 15 - Validation loss: 251.14069112141928
Epoch 16 - Validation loss: 247.81373596191406
Epoch 17 - Validation loss: 247.78069814046225
Epoch 18 - Validation loss: 247.7538536919488
Ep

In [72]:
model.eval()
model(torch.tensor([[0,1],[0,2],[0,3],[0,4],[0,5],[0,7],[0,9],[0,11]]))
model(torch.tensor([[0,5,1],[0,5,2],[0,5,3],[0,5,4],[0,5,5],[0,5,7],[0,5,9],[0,5,11]]))
# model(torch.tensor([[0,5,4,1],[0,5,4,2],[0,5,4,3],[0,5,4,4],[0,5,4,5],[0,5,4,7],[0,5,4,9],[0,5,4,11]]))

tensor([[[172.6630],
         [172.4937],
         [173.4037]],

        [[172.6630],
         [172.4937],
         [172.5397]],

        [[172.6630],
         [172.4937],
         [172.9803]],

        [[172.6630],
         [172.4937],
         [168.3261]],

        [[172.6630],
         [172.4937],
         [172.6562]],

        [[172.6630],
         [172.4937],
         [174.0859]],

        [[172.6630],
         [172.4937],
         [167.2914]],

        [[172.6630],
         [172.4937],
         [165.5466]]], grad_fn=<ViewBackward0>)